<a href="https://colab.research.google.com/github/earlbryt/Todo-List/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x



Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [71]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import clear_output




In [72]:
trainx = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
evalx = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = trainx.pop('survived')
y_eval = evalx.pop('survived')

In [73]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']

NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = trainx[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [74]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(trainx, y_train)
eval_input_fn = make_input_fn(evalx, y_eval, num_epochs=1, shuffle=False)


In [75]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [76]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
# print(result['accuracy'])
print(result)

{'accuracy': 0.7651515, 'accuracy_baseline': 0.625, 'auc': 0.83654726, 'auc_precision_recall': 0.7699792, 'average_loss': 0.51559645, 'label/mean': 0.375, 'loss': 0.50562835, 'precision': 0.8135593, 'prediction/mean': 0.27535093, 'recall': 0.4848485, 'global_step': 200}


In [89]:
result = list(linear_est.predict(eval_input_fn))
print(y_eval.loc[4])
print(result[4]['probabilities'][0])

1
0.8833844
